In [58]:
import pandas as pd

def students_and_examinations(students: pd.DataFrame, subjects: pd.DataFrame, examinations: pd.DataFrame) -> pd.DataFrame:
    """
    这个是真不会, 等我看看题解
    """
    new_df = students.merge(examinations, how="left")
    new_df["attended_exams"] = 0
    new_df = new_df.groupby(["student_id", "student_name", "subject_name"]).count().reset_index()
    
    df2 = students.merge(subjects, how="cross")
    df2 = df2.merge(new_df, how="left").fillna(0)

    # 需要排序
    df2 = df2.sort_values(by=["student_id", "subject_name"])
    return df2
    

In [2]:
data = [[1, 'Alice'], [2, 'Bob'], [13, 'John'], [6, 'Alex']]
Students = pd.DataFrame(data, columns=['student_id', 'student_name']).astype({'student_id':'Int64', 'student_name':'object'})
data = [['Math'], ['Physics'], ['Programming']]
Subjects = pd.DataFrame(data, columns=['subject_name']).astype({'subject_name':'object'})
data = [[1, 'Math'], [1, 'Physics'], [1, 'Programming'], [2, 'Programming'], [1, 'Physics'], [1, 'Math'], [13, 'Math'], [13, 'Programming'], [13, 'Physics'], [2, 'Math'], [1, 'Math']]
Examinations = pd.DataFrame(data, columns=['student_id', 'subject_name']).astype({'student_id':'Int64', 'subject_name':'object'})

In [3]:
Students

,student_id,student_name
0,1,Alice
1,2,Bob
2,13,John
3,6,Alex


In [4]:
Subjects

,subject_name
0,Math
1,Physics
2,Programming


In [5]:
Examinations

,student_id,subject_name
0,1,Math
1,1,Physics
2,1,Programming
3,2,Programming
4,1,Physics
5,1,Math
6,13,Math
7,13,Programming
8,13,Physics
9,2,Math


In [59]:
students_and_examinations(Students, Subjects, Examinations)

,student_id,student_name,subject_name,attended_exams
0,1,Alice,Math,3.0
1,1,Alice,Physics,2.0
2,1,Alice,Programming,1.0
3,2,Bob,Math,1.0
4,2,Bob,Physics,0.0
5,2,Bob,Programming,1.0
9,6,Alex,Math,0.0
10,6,Alex,Physics,0.0
11,6,Alex,Programming,0.0
6,13,John,Math,1.0


In [9]:
Students.merge(Subjects, how="cross")

,student_id,student_name,subject_name
0,1,Alice,Math
1,1,Alice,Physics
2,1,Alice,Programming
3,2,Bob,Math
4,2,Bob,Physics
5,2,Bob,Programming
6,13,John,Math
7,13,John,Physics
8,13,John,Programming
9,6,Alex,Math


In [11]:
Students.merge(Subjects, how="cross").merge(Examinations, how="left")

,student_id,student_name,subject_name
0,1,Alice,Math
1,1,Alice,Math
2,1,Alice,Math
3,1,Alice,Physics
4,1,Alice,Physics
5,1,Alice,Programming
6,2,Bob,Math
7,2,Bob,Physics
8,2,Bob,Programming
9,13,John,Math


In [32]:
Students.merge(Examinations, how="left")

,student_id,student_name,subject_name
0,1,Alice,Math
1,1,Alice,Physics
2,1,Alice,Programming
3,1,Alice,Physics
4,1,Alice,Math
5,1,Alice,Math
6,2,Bob,Programming
7,2,Bob,Math
8,13,John,Math
9,13,John,Programming


In [34]:
new_df = Students.merge(Examinations, how="left")
new_df["val"] = 0
new_df = new_df.groupby(["student_id", "student_name", "subject_name"]).count().reset_index()
new_df

,student_id,student_name,subject_name,val
0,1,Alice,Math,3
1,1,Alice,Physics,2
2,1,Alice,Programming,1
3,2,Bob,Math,1
4,2,Bob,Programming,1
5,13,John,Math,1
6,13,John,Physics,1
7,13,John,Programming,1


In [53]:
df2 = Students.merge(Subjects, how="cross")
df2 = df2.merge(new_df, how="left").fillna(0)
df2

,student_id,student_name,subject_name,val
0,1,Alice,Math,3.0
1,1,Alice,Physics,2.0
2,1,Alice,Programming,1.0
3,2,Bob,Math,1.0
4,2,Bob,Physics,0.0
5,2,Bob,Programming,1.0
6,13,John,Math,1.0
7,13,John,Physics,1.0
8,13,John,Programming,1.0
9,6,Alex,Math,0.0


# 题解

In [63]:
grouped = Examinations.groupby(['student_id', 'subject_name']).size().reset_index(name='attended_exams')
grouped

,student_id,subject_name,attended_exams
0,1,Math,3
1,1,Physics,2
2,1,Programming,1
3,2,Math,1
4,2,Programming,1
5,13,Math,1
6,13,Physics,1
7,13,Programming,1


In [64]:
all_id_subjects = pd.merge(Students, Subjects, how='cross')
all_id_subjects

,student_id,student_name,subject_name
0,1,Alice,Math
1,1,Alice,Physics
2,1,Alice,Programming
3,2,Bob,Math
4,2,Bob,Physics
5,2,Bob,Programming
6,13,John,Math
7,13,John,Physics
8,13,John,Programming
9,6,Alex,Math


In [65]:
id_subjects_count = pd.merge(all_id_subjects, grouped, on=['student_id', 'subject_name'], how='left')
id_subjects_count

,student_id,student_name,subject_name,attended_exams
0,1,Alice,Math,3.0
1,1,Alice,Physics,2.0
2,1,Alice,Programming,1.0
3,2,Bob,Math,1.0
4,2,Bob,Physics,NaN
5,2,Bob,Programming,1.0
6,13,John,Math,1.0
7,13,John,Physics,1.0
8,13,John,Programming,1.0
9,6,Alex,Math,NaN


In [66]:
id_subjects_count['attended_exams'] = id_subjects_count['attended_exams'].fillna(0).astype(int)
id_subjects_count

,student_id,student_name,subject_name,attended_exams
0,1,Alice,Math,3
1,1,Alice,Physics,2
2,1,Alice,Programming,1
3,2,Bob,Math,1
4,2,Bob,Physics,0
5,2,Bob,Programming,1
6,13,John,Math,1
7,13,John,Physics,1
8,13,John,Programming,1
9,6,Alex,Math,0
